In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

sales_ci=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\uncertainty_quant.xlsx", sheet_name='sales_ci')
unc=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\uncertainty_quant.xlsx", sheet_name='unc')
unc_desc=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\uncertainty_quant.xlsx", sheet_name='unc_desc')


sales_ci=sales_ci.melt(id_vars=['product', 'region', 'units', 'project'], 
        var_name="year", 
        value_name="sales")
sales_ci=sales_ci[(sales_ci['units'] =='¥') & (sales_ci['project']=='Total')]

# Create Output Dataframe
output_df = pd.DataFrame(columns=['product', 'region', 'units', 'project', 'year', 'sales', 'sales_RA_10', 'sales_RA_50', 'sales_RA_90'])

sales_ci

,product,region,units,project,year,sales
10,xtandi,US,¥,Total,2022,3490.227914
21,xtandi,JP,¥,Total,2022,438.632820
32,xtandi,CN,¥,Total,2022,67.308338
43,xtandi,DE,¥,Total,2022,518.145979
54,xtandi,FR,¥,Total,2022,260.011082
...,...,...,...,...,...,...
2869,zolbe,GCN,¥,Total,2025,0.000000
2880,zolbe,WW,¥,Total,2025,361.337003
2881,tacrolimus,WW,¥,Total,2025,1585.738822
2882,lexiscan,WW,¥,Total,2025,16.144346


In [2]:
#############
# XTANDI
#############
years = [2022, 2023, 2024, 2025]
product = 'xtandi'

for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Uncertainty Probability (value)
        unc1_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc1')]['probability'], 1, 1)
        unc2_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc2')]['probability'], 1, 1)
        unc3_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc3')]['probability'], 1, 1)

        # Get Uncertainty Quant by year (series)
        unc1=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc1']
        unc2=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc2']
        unc3=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc3']

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=int(sales['sales']) + int(unc1*unc1_prob) + int(unc2*unc2_prob) + int(unc3*unc3_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)
    
output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.9,6221.0,6264.0
1,xtandi,WW,¥,Total,2023,6574.134521,6539.9,6584.0,6640.1
2,xtandi,WW,¥,Total,2024,6902.520169,6880.0,6946.5,7033.1
3,xtandi,WW,¥,Total,2025,7310.662392,7305.8,7397.0,7515.2


In [3]:
print(sum(output_df[(output_df['product'] =='xtandi')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='xtandi')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='xtandi')]['sales_RA_90']))              

26914.6
27148.5
27452.4


In [4]:
#############
# XOSPATA
#############
years = [2022, 2023, 2024, 2025]
product = 'xospata'


for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Uncertainty Probability (value)
        unc4_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc4')]['probability'], 1, 1)
        unc5_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc5')]['probability'], 1, 1)
        unc6_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc6')]['probability'], 1, 1)
        
        # Get relevant sales values for uncertainties
        unc4_sales=sum(sales_ci[(sales_ci['product']==product) & (sales_ci['year'] ==year) & sales_ci['region'].isin(['US', 'JP', 'DE', 'FR', 'IT', 'ES', 'GB'])]['sales'])
        unc5_sales=sum(sales_ci[(sales_ci['product']==product) & (sales_ci['year'] ==year) & sales_ci['region'].isin(['US', 'DE', 'FR', 'IT', 'ES', 'GB'])]['sales'])
        unc6_sales=sum(sales_ci[(sales_ci['product']==product) & (sales_ci['year'] ==year) & sales_ci['region'].isin(['US', 'DE', 'FR', 'IT', 'ES', 'GB'])]['sales'])

        # Get Uncertainty Quant by year (series)
        unc4=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc4']*unc4_sales
        unc5=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc5']*unc5_sales
        unc6=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc6']*unc6_sales

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=int(sales['sales']) + int(unc4*unc4_prob) + int(unc5*unc5_prob) + int(unc6*unc6_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)
    
output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.9,6221.0,6264.0
1,xtandi,WW,¥,Total,2023,6574.134521,6539.9,6584.0,6640.1
2,xtandi,WW,¥,Total,2024,6902.520169,6880.0,6946.5,7033.1
3,xtandi,WW,¥,Total,2025,7310.662392,7305.8,7397.0,7515.2
4,xospata,WW,¥,Total,2022,447.374502,424.0,429.0,436.0
5,xospata,WW,¥,Total,2023,533.562057,507.0,514.0,520.0
6,xospata,WW,¥,Total,2024,607.117290,578.0,585.0,593.0
7,xospata,WW,¥,Total,2025,767.757646,728.0,738.0,749.0


In [5]:
print(sum(output_df[(output_df['product'] =='xospata')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='xospata')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='xospata')]['sales_RA_90']))   

2237.0
2266.0
2298.0


In [6]:
#############
# PADCEV
#############
years = [2022, 2023, 2024, 2025]
product = 'padcev'

for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Uncertainty Probability (value)
        unc7_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc7')]['probability'], 1, 1)
        unc8_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc8')]['probability'], 1, 1)
        unc9_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc9')]['probability'], 1, 1)
        unc10_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc10')]['probability'], 1, 1)
        unc11_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc11')]['probability'], 1, 1)
        unc12_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc12')]['probability'], 1, 1)
        unc13_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc13')]['probability'], 1, 1)
        unc14_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc14')]['probability'], 1, 1)

        # Get Uncertainty Quant by year (series)
        unc7=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc7']
        unc8=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc8']
        unc9=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc9']
        unc10=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc10']
        unc11=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc11']
        unc12=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc12']
        unc13=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc13']
        unc14=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc14']

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=int(sales['sales']) + int(unc7*unc7_prob) + int(unc8*unc8_prob) + int(unc9*unc9_prob) + int(unc10*unc10_prob) + int(unc11*unc11_prob) + int(unc12*unc12_prob) + int(unc13*unc13_prob) + int(unc14*unc14_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)
    
output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.9,6221.0,6264.0
1,xtandi,WW,¥,Total,2023,6574.134521,6539.9,6584.0,6640.1
2,xtandi,WW,¥,Total,2024,6902.520169,6880.0,6946.5,7033.1
3,xtandi,WW,¥,Total,2025,7310.662392,7305.8,7397.0,7515.2
4,xospata,WW,¥,Total,2022,447.374502,424.0,429.0,436.0
5,xospata,WW,¥,Total,2023,533.562057,507.0,514.0,520.0
6,xospata,WW,¥,Total,2024,607.117290,578.0,585.0,593.0
7,xospata,WW,¥,Total,2025,767.757646,728.0,738.0,749.0
8,padcev,WW,¥,Total,2022,542.696779,551.0,557.0,562.0
9,padcev,WW,¥,Total,2023,798.444301,798.0,806.5,821.0


In [7]:
print(sum(output_df[(output_df['product'] =='padcev')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='padcev')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='padcev')]['sales_RA_90']))

4096.9
4142.5
4201.1


In [8]:
#############
# EVRENZO
#############
years = [2022, 2023, 2024, 2025]
product = 'evrenzo'

for year in years:
    print(year)
    sales_ra = []
    n=500
    
    for i in range(n):
        # Get Uncertainty Probability (value)
        unc15_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc15')]['probability'], 1, 1)
        unc16_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc16')]['probability'], 1, 1)
        unc17_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc17')]['probability'], 1, 1)
        unc18_prob=np.random.triangular(0, unc_desc[(unc_desc['uncertainties'] == 'unc18')]['probability'], 1, 1)

        # Get Uncertainty Quant by year (series)
        unc15=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc15']
        unc16=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc16']
        unc17=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc17']
        unc18=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc18']

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=int(sales['sales']) + int(unc15*unc15_prob) + int(unc16*unc16_prob) + int(unc17*unc17_prob) + int(unc18*unc18_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)
    
output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.9,6221.0,6264.0
1,xtandi,WW,¥,Total,2023,6574.134521,6539.9,6584.0,6640.1
2,xtandi,WW,¥,Total,2024,6902.520169,6880.0,6946.5,7033.1
3,xtandi,WW,¥,Total,2025,7310.662392,7305.8,7397.0,7515.2
4,xospata,WW,¥,Total,2022,447.374502,424.0,429.0,436.0
5,xospata,WW,¥,Total,2023,533.562057,507.0,514.0,520.0
6,xospata,WW,¥,Total,2024,607.117290,578.0,585.0,593.0
7,xospata,WW,¥,Total,2025,767.757646,728.0,738.0,749.0
8,padcev,WW,¥,Total,2022,542.696779,551.0,557.0,562.0
9,padcev,WW,¥,Total,2023,798.444301,798.0,806.5,821.0


In [9]:
print(sum(output_df[(output_df['product'] =='evrenzo')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='evrenzo')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='evrenzo')]['sales_RA_90']))

1081.0
1104.0
1123.1


In [10]:
#############
# FEZO
#############
years = [2022, 2023, 2024, 2025]
product = 'fezo'

for year in years:
    print(year)

    fezo_base=sales_ci[(sales_ci['product']==product) & (sales_ci['region']=='US') & (sales_ci['year']==year)]['sales']
    fezo_10 = float(unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc19'])
    fezo_50 = float(unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc20'])
    fezo_90 = float(unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc21'])

    # Get Product Worldwide sales by year (series) 
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = float(fezo_10/fezo_base)*float(prod_ww_sales['sales'])
    prod_ww_sales['sales_RA_50'] = float(fezo_50/fezo_base)*float(prod_ww_sales['sales'])
    prod_ww_sales['sales_RA_90'] = float(fezo_90/fezo_base)*float(prod_ww_sales['sales'])

    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [11]:
print(sum(output_df[(output_df['product'] =='fezo')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='fezo')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='fezo')]['sales_RA_90']))   

nan
nan
nan


In [12]:
#############
# MIRABEGRON
#############
product = 'mirabegron'

year=2022
prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]

prod_ww_sales['sales_RA_10'] = 1723
prod_ww_sales['sales_RA_50'] = 1723
prod_ww_sales['sales_RA_90'] = 1723

output_df = output_df.append(prod_ww_sales, ignore_index=True)

year=2023
prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]

prod_ww_sales['sales_RA_10'] = 1708
prod_ww_sales['sales_RA_50'] = 1713
prod_ww_sales['sales_RA_90'] = 1713

output_df = output_df.append(prod_ww_sales, ignore_index=True)

year=2024
prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]

prod_ww_sales['sales_RA_10'] = 944
prod_ww_sales['sales_RA_50'] = 1686
prod_ww_sales['sales_RA_90'] = 1686

output_df = output_df.append(prod_ww_sales, ignore_index=True)

year=2025
prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]

prod_ww_sales['sales_RA_10'] = 731
prod_ww_sales['sales_RA_50'] = 1190
prod_ww_sales['sales_RA_90'] = 1672

output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [13]:
print(sum(output_df[(output_df['product'] =='mirabegron')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='mirabegron')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='mirabegron')]['sales_RA_90']))  

5106.0
6312.0
6794.0


In [14]:
#############
# ZOLBE
#############
years = [2022, 2023, 2024, 2025]
product = 'zolbe'

for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=float(sales['sales']) + float(sales['sales']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [15]:
print(sum(output_df[(output_df['product'] =='zolbe')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='zolbe')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='zolbe')]['sales_RA_90']))  

417.9
473.5
529.1


In [16]:
#############
# TACROLIMUS
#############
years = [2022, 2023, 2024, 2025]
product = 'tacrolimus'

for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=float(sales['sales']) + float(sales['sales']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [17]:
print(sum(output_df[(output_df['product'] =='tacrolimus')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='tacrolimus')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='tacrolimus')]['sales_RA_90']))  

6331.9
6601.5
6863.1


In [41]:
#############
# LEXISCAN
#############
years = [2022, 2023, 2024, 2025]
product = 'lexiscan'

for year in years:
    print(year)
    n=5
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=float(sales['sales']) + float(sales['sales']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [19]:
print(sum(output_df[(output_df['product'] =='lexiscan')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='lexiscan')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='lexiscan')]['sales_RA_90']))  

1126.9
1174.0
1220.1


In [20]:
#############
# OTHERS
#############
years = [2022, 2023, 2024, 2025]
product = 'others'

for year in years:
    print(year)
    n=500
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc[(unc['project'] ==product) & (unc['year'] ==year)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
        unc_all=float(sales['sales']) + float(sales['sales']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_ci[(sales_ci['product'] ==product) & (sales_ci['region']=='WW') & (sales_ci['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

output_df

2022
2023
2024
2025


,product,region,units,project,year,sales,sales_RA_10,sales_RA_50,sales_RA_90
0,xtandi,WW,¥,Total,2022,6223.689687,6188.900000,6221.000000,6264.000000
1,xtandi,WW,¥,Total,2023,6574.134521,6539.900000,6584.000000,6640.100000
2,xtandi,WW,¥,Total,2024,6902.520169,6880.000000,6946.500000,7033.100000
3,xtandi,WW,¥,Total,2025,7310.662392,7305.800000,7397.000000,7515.200000
4,xospata,WW,¥,Total,2022,447.374502,424.000000,429.000000,436.000000
5,xospata,WW,¥,Total,2023,533.562057,507.000000,514.000000,520.000000
6,xospata,WW,¥,Total,2024,607.117290,578.000000,585.000000,593.000000
7,xospata,WW,¥,Total,2025,767.757646,728.000000,738.000000,749.000000
8,padcev,WW,¥,Total,2022,542.696779,551.000000,557.000000,562.000000
9,padcev,WW,¥,Total,2023,798.444301,798.000000,806.500000,821.000000


In [21]:
print(sum(output_df[(output_df['product'] =='others')]['sales_RA_10']))
print(sum(output_df[(output_df['product'] =='others')]['sales_RA_50']))
print(sum(output_df[(output_df['product'] =='others')]['sales_RA_90']))  

9026.9
9785.0
10561.5


In [37]:
# Analysis
output_df.fillna(0)

strategic_products = ['xtandi', 'xospata', 'padcev', 'evrenzo', 'fezo', 'zolbe']

print('\n')
print('Sales 10th')
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xtandi')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xospata')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='padcev')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='evrenzo')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='fezo')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='zolbe')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product'].isin(strategic_products))]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='mirabegron')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='tacrolimus')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='lexiscan')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='others')]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_10'])))

print('\n')
print('Sales 50th')
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xtandi')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xospata')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='padcev')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='evrenzo')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='fezo')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='zolbe')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product'].isin(strategic_products))]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='mirabegron')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='tacrolimus')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='lexiscan')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='others')]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_50'])))

print('\n')
print('Sales 90th')
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xtandi')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xospata')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='padcev')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='evrenzo')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='fezo')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='zolbe')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product'].isin(strategic_products))]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='mirabegron')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='tacrolimus')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='lexiscan')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='others')]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_90'])))

print('\n')
print('Sales')
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xtandi')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='xospata')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='padcev')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='evrenzo')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='fezo')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='zolbe')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product'].isin(strategic_products))]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='mirabegron')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='tacrolimus')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='lexiscan')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025) & (output_df['product']=='others')]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales'])))



Sales 10th
7306
728
1669
430
1391
318
11842
731
1520
16
2262
16371


Sales 50th
7397
738
1689
442
2240
361
12867
1190
1584
16
2442
18100


Sales 90th
7515
749
1714
452
3583
404
14417
1672
1646
16
2644
20395


Sales
7311
768
1662
447
2633
361
13182
1190
1586
16
2454
18428


In [43]:
print(round(sum(output_df[(output_df['year'] ==2023)]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2024)]['sales_RA_10'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_10'])))

15867
14833
16435


In [44]:
print(round(sum(output_df[(output_df['year'] ==2023)]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2024)]['sales_RA_50'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_50'])))

16484
16571
18164


In [45]:
print(round(sum(output_df[(output_df['year'] ==2023)]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2024)]['sales_RA_90'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales_RA_90'])))

17247
17965
20460


In [46]:
print(round(sum(output_df[(output_df['year'] ==2023)]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2024)]['sales'])))
print(round(sum(output_df[(output_df['year'] ==2025)]['sales'])))

16591
16841
18493
